## Vector Search in Oracle DB 23ai

- logging: Pythonの標準ライブラリでログ記録のため
- os: 環境変数を操作するために使用
- sys: Pythonのシステム固有のパラメータや関数にアクセスするために使用
- dotenv.load_dotenv と dotenv.find_dotenv: python-dotenvパッケージの関数で、.envファイルから環境変数を読み込み

In [1]:
import logging
import os
import sys
from dotenv import load_dotenv, find_dotenv

- .envファイルから環境変数を読み込んで、プログラム内で使用できるようにします。
- logger = logging.getLogger(__name__): 現在のモジュール名を使用してロガーオブジェクトを作成します。
- handler = logging.StreamHandler(sys.stdout): 標準出力（コンソール）にログメッセージを送るストリームハンドラを作成します。

- log_level = os.getenv("LOG_LEVEL", "ERROR").upper(): 環境変数 LOG_LEVEL からログレベルを取得します。LOG_LEVEL が設定されていない場合はデフォルトで "ERROR" になります。upper() メソッドでログレベルを大文字に変換します。
- handler.setLevel(log_level): ハンドラのログレベルを設定します。
- logger.setLevel(log_level): ロガーのログレベルを設定します。

In [2]:
_ = load_dotenv(find_dotenv())

logger = logging.getLogger(__name__)
handler = logging.StreamHandler(sys.stdout)
log_level = os.getenv("LOG_LEVEL", "ERROR").upper()
handler.setLevel(log_level)
logger.setLevel(log_level)

- glob: ファイルパスのパターンマッチングを行うために使用されます。
- Docx2txtLoader: .docx ファイルを読み込むためのクラスで、langchain_community.document_loaders モジュールからインポートします。

In [3]:
import glob
from langchain_community.document_loaders import Docx2txtLoader

../data/ ディレクトリ内のすべての .docx ファイルを取得

In [4]:
files = glob.glob("./data/*.docx")

ドキュメントの初期化

In [5]:
documents = []

### ファイルの読み込みとログ出力:

for file in files: 取得したファイルリストをループで処理します。
- logger.debug(f"loaded file name: {file}"): 現在処理中のファイル名をデバッグログに出力します。
- loader = Docx2txtLoader(file): Docx2txtLoader のインスタンスを作成し、ファイルをロードします。
- document = loader.load(): ファイルの内容を読み込みます。
- logger.debug(f"content: {document}"): 読み込んだファイルの内容をデバッグログに出力します。
- documents.extend(document): 読み込んだドキュメントを documents リストに追加します。

In [6]:
for file in files:
    logger.debug(f"loaded file name: {file}")
    loader = Docx2txtLoader(file)
    document = loader.load()
    logger.debug(f"content: {document}")
    documents.extend(document)


In [7]:
logger.info(f"documents: {documents}")

### 読み込んだドキュメントをチャンク分割

In [8]:
import oracledb
from langchain_community.vectorstores.oraclevs import OracleVS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_community.embeddings.oci_generative_ai import OCIGenAIEmbeddings
from langchain_community.document_loaders.oracleai import OracleTextSplitter

In [9]:
# for Oracle Database 23ai
username = os.getenv("USERNAME")
password = os.getenv("PASSWORD")
dsn = os.getenv("DSN")
config_dir = os.getenv("CONFIG_DIR")
wallet_dir = os.getenv("WALLET_DIR")
wallet_password = os.getenv("WALLET_PASSWORD")
table_name = os.getenv("TABLE_NAME")

logger.debug(f"username: {username}")
logger.debug(f"password: {password}")
logger.debug(f"dsn: {dsn}")
logger.debug(f"wallet dir: {wallet_dir}")
logger.debug(f"wallet password: {wallet_password}")
logger.debug(f"table name: {table_name}")

# for OCI Generative AI Service
compartment_id = os.getenv("COMPARTMENT_ID")
service_endpoint = os.getenv("SERVICE_ENDPOINT")

logger.debug(f"compartment id: {compartment_id}")
logger.debug(f"service endpoint: {service_endpoint}")


2024-07-22 05:23:07,879 - DEBUG - username: vector
2024-07-22 05:23:07,881 - DEBUG - password: Welcome123##
2024-07-22 05:23:07,885 - DEBUG - dsn: lbr4os9vcq1jmoml_high
2024-07-22 05:23:07,885 - DEBUG - wallet dir: /home/opc/network/admin
2024-07-22 05:23:07,886 - DEBUG - wallet password: Welcome123##
2024-07-22 05:23:07,887 - DEBUG - table name: rule
2024-07-22 05:23:07,888 - DEBUG - compartment id: ocid1.compartment.oc1..aaaaaaaaf47nqirak4i2wfp3t4sepz4y6dl2pbuv2iv767mteqpalwfidbpq
2024-07-22 05:23:07,890 - DEBUG - service endpoint: https://inference.generativeai.us-chicago-1.oci.oraclecloud.com


In [10]:
connection = oracledb.connect(
    dsn=dsn,
    user=username,
    password=password,
    config_dir=config_dir,
    wallet_location=wallet_dir,
    wallet_password=wallet_password
)

In [11]:
embedding_function = OCIGenAIEmbeddings(
    auth_type="INSTANCE_PRINCIPAL",
    model_id="cohere.embed-multilingual-v3.0",
    service_endpoint=service_endpoint,
    compartment_id=compartment_id,
)

2024-07-22 05:23:08,126 - DEBUG - Starting new HTTP connection (1): 169.254.169.254:80
2024-07-22 05:23:08,133 - DEBUG - http://169.254.169.254:80 "GET /opc/v2/identity/cert.pem HTTP/1.1" 200 None
2024-07-22 05:23:08,143 - DEBUG - http://169.254.169.254:80 "GET /opc/v2/identity/key.pem HTTP/1.1" 200 1679
2024-07-22 05:23:08,187 - DEBUG - Starting new HTTP connection (1): 169.254.169.254:80
2024-07-22 05:23:08,193 - DEBUG - http://169.254.169.254:80 "GET /opc/v2/identity/intermediate.pem HTTP/1.1" 200 None
2024-07-22 05:23:08,283 - DEBUG - Starting new HTTP connection (1): 169.254.169.254:80
2024-07-22 05:23:08,291 - DEBUG - http://169.254.169.254:80 "GET /opc/v2/instance/region HTTP/1.1" 200 10
2024-07-22 05:23:08,314 - DEBUG - http://169.254.169.254:80 "GET /opc/v2/identity/cert.pem HTTP/1.1" 200 None
2024-07-22 05:23:08,325 - DEBUG - http://169.254.169.254:80 "GET /opc/v2/identity/key.pem HTTP/1.1" 200 1679
2024-07-22 05:23:08,375 - DEBUG - http://169.254.169.254:80 "GET /opc/v2/iden

### 埋め込み関数の設定
- auth_type="INSTANCE_PRINCIPAL": インスタンスプリンシパル認証を使用してOCIにアクセスします。
- model_id="cohere.embed-multilingual-v3.0": Cohereの多言語埋め込みモデルを指定しています。
- service_endpoint: OCIのジェネレーティブ AI サービスのエンドポイント。
- compartment_id: OCI コンパートメント ID。

In [13]:

oracle_vs = OracleVS(
    client=connection,
    embedding_function=embedding_function,
    table_name=table_name,
    distance_strategy=DistanceStrategy.COSINE,
    query="What is Oracle Database?"
)

2024-07-22 05:23:08,725 - DEBUG - Starting new HTTPS connection (1): inference.generativeai.us-chicago-1.oci.oraclecloud.com:443
2024-07-22 05:23:09,973 - DEBUG - https://inference.generativeai.us-chicago-1.oci.oraclecloud.com:443 "POST /20231130/actions/embedText HTTP/1.1" 200 5285
2024-07-22 05:23:10,197 - INFO - Table already exists...


### Oracle Vector Store の設定
- client=connection: Oracle データベースへの接続。
- embedding_function=embedding_function: 前述の埋め込み関数。
- table_name=table_name: ベクトル検索を行うテーブルの名前。
- distance_strategy=DistanceStrategy.COSINE: コサイン距離を使用したベクトル比較戦略。
- query="What is Oracle Database?": 実行するクエリ。


In [14]:
splitter_params = {"split": "recursively", "max": 300, "by": "words", "overlap": 30, "normalize": "all"}
splitter = OracleTextSplitter(conn=connection, params=splitter_params)

data = splitter.split_documents(documents=documents)

### Oracle Text Spliter
1. Spliterのパラメータと接続情報を渡して分割
2. dataに格納

In [15]:
oracle_vs.add_documents(documents=data)

2024-07-22 05:23:10,644 - DEBUG - https://inference.generativeai.us-chicago-1.oci.oraclecloud.com:443 "POST /20231130/actions/embedText HTTP/1.1" 200 79386


['4B956415826EBC09',
 '3B48B888AA8E7042',
 '09F83738B4EF51FB',
 '7D8C21918FC3F911',
 'B36D92A729C1C4E4',
 '66D616031E1C5965',
 '0B97B3014278BEBF',
 'F2382A31E0846FF4',
 '92F8E3F23053736E',
 '95FC1ECD4BAE32C7',
 'ABD0B3D16DDCDB3F',
 '4B71EF05F3151F66',
 '850E7E275D52A326',
 'D764C4DAA147648D',
 'EB4823FED80032C2',
 'C5596A81FE265C34',
 '34301399CAF4406F',
 '7F4A579110BD1CA8']

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models.oci_generative_ai import ChatOCIGenAI

chat = ChatOCIGenAI(
    auth_type="INSTANCE_PRINCIPAL",
    service_endpoint=service_endpoint,
    compartment_id=compartment_id,
    model_id="cohere.command-r-plus",
    is_stream=False,
    model_kwargs={
        "temperature": 0,
        "max_tokens": 500,
        "top_p": 0.75,
        "top_k": 0,
        "frequency_penalty": 0,
        "presence_penalty": 0
    }
)

template = """
可能な限り、検索によって得られたコンテキストに則って回答を作成してください。
コンテキスト: {context}
---
質問: {query}
"""

prompt_template = PromptTemplate.from_template(
    template=template,
)

chain = (
    {"context": oracle_vs.as_retriever(), "query": RunnablePassthrough()}
    | prompt_template
    | chat
    | StrOutputParser()
)

# ストリーム出力の代わりにinvoke()を使用
res = chain.invoke("海外出張って日当とかでるんでしたっけ？")

print(res)
